<a href="https://colab.research.google.com/github/michalis0/Business-Intelligence-and-Analytics/blob/master/week5%20-%20SQL/exercices/SQL1_EX2_Solutions.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Week 5 - SQL (1/2) - Exercice 2

# National Collegiate Athletic Association | NCAA
We will now use an extract of a dataset about the NCAA, which oversees and regulates North American university sports.

College presidents across the USA recognized a need to track how student-athletes are doing academically prior to graduation. Therefore, they implemeted a comprehensive academic refore package designed to improve the academic success and graduation of all student-athletes. The centerpiece of the academic reform package was the development of a real-time academic measurement for sports teams, known as the Academic Progress Rate (APR).

The APR includes student-athlete eligibility, retention and graduation as factors in a formula that yields a single number, providing a much clearer picture of the current academic culture on each sports team. Since its inception, the APR has become an important measure of student-athlete academic success. Additionally, student-athletes can be ruled ineligible to play if they demonstrate poor academic performances.

In the extract of the data we will be using, we will compare teams based on their number of student-athletes on the roster (`(Previous)Athletes`), their APR score (`(Previous)Score`), and their ratio of eligible players (`(previous)Eligibility`). We have these information for both the "current" year (2014) and "two" years back (2012).

[(data source)](https://www.kaggle.com/ncaa/academic-scores)

In [ ]:
import pandas as pd
from sqlalchemy import create_engine
db = create_engine('sqlite://', echo=False)
csvfile = 'https://raw.githubusercontent.com/michalis0/Business-Intelligence-and-Analytics/master/data/NCAA.csv'
df = pd.read_csv(csvfile, delimiter=';')
table_name = 'ncaa'
df.to_sql(table_name, con=db)

Now, let's take a first look at the data we will be working with.

In [ ]:
query = """
select *
from ncaa
limit 5
"""
sql_df = pd.read_sql(query, con=db)
sql_df

,index,School,Sport,Conference,Athletes,Score,Eligibility,PreviousAthletes,PreviousScore,PreviousEligibility
0,0,Northwestern University,Men's Basketball,Big Ten Conference,12,932,0.9565,12.0,979.0,0.9583
1,1,University of Oregon,Men's Basketball,Pac-12 Conference,13,905,0.9565,13.0,952.0,1.0000
2,2,Dartmouth College,Women's Basketball,The Ivy League,15,1000,1.0000,10.0,1000.0,1.0000
3,3,Texas Christian University,Men's Basketball,Big 12 Conference,14,926,0.9259,13.0,918.0,0.8800
4,4,University of Louisville,Women's Basketball,Atlantic Coast Conference,14,1000,1.0000,17.0,1000.0,1.0000


Each row contains a team. We can see that the dataset contains the following information:
- **`School`:** Name of the university affiliated to the team.
- **`Sport`:** Name of the sport (and gender) the team plays.
- **`Conference`:** Conference the team is playing in.
- **`Athletes`:** Number of athletes currently on the team (2014).
- **`Score`:**  Current Academic Progress Rate (APR) of the team (2014).
- **``Eligibility:``** Ratio of players currently eligible to play (2014).
- **`PreviousAthletes`:** Number of athletes previously on the team (2012).
- **`PreviousScore`:**  Previous Academic Progress Rate (APR) of the team (2012).
- **``PreviousEligibility:``** Ratio of players previously eligible to play (2012).

### Question 1
Find all the schools whose name contains a `W` and have an ice-hockey team.

Hint: To account for both women and men's teams, use `'%Ice Hockey'`.

In [ ]:
query = """
select distinct School
from ncaa
where School like '%W%' and Sport like '%Ice Hockey'
"""
sql_df = pd.read_sql(query, con=db)
sql_df

,School
0,"University of Minnesota, Twin Cities"
1,"University of Wisconsin, Madison"
2,Brown University


### Question 2
Select the worst team that received the worst `Score`, as well as that `Score`.

Hint: You need to select the `School`, the `Sport`, and the `Score` as `WorstScore`.

In [ ]:
query = """
select School, Sport, min(Score) as WorstScore
from ncaa
"""
sql_df = pd.read_sql(query, con=db)
sql_df

,School,Sport,WorstScore
0,University of South Dakota,Women's Tennis,875


### Question 3
Find the three schools whose athletes have the worst `Score` on average.

Hint: Use `order by` and `limit`.

In [ ]:
query = """
select School, avg(Score) as AverageScore
from ncaa
group by School
order by AverageScore asc
limit 3
"""
sql_df = pd.read_sql(query, con=db)
sql_df

,School,AverageScore
0,University of South Dakota,941.0
1,Georgia Institute of Technology,955.0
2,Oregon State University,963.0


### Questions 4
Find the three conferences with the most schools in them. Display both the name of the conferences and the respective number of schools they contain (as `NumberOfSchools`).

Hint: You need to count the amount of distinct schools in each conference.

In [ ]:
query = """
select Conference, count(distinct School) as NumberOfSchools
from ncaa
group by Conference
order by NumberOfSchools desc
limit 3
"""
sql_df = pd.read_sql(query, con=db)
sql_df

,Conference,NumberOfSchools
0,Atlantic Coast Conference,15
1,Big Ten Conference,14
2,Pac-12 Conference,12


### Question 5
How many `PreviousAthletes` that played in teams where every athlete was eligible (`PreviousEligibility = 1`) did each schools of the `The Ivy League` conference have? Additionally, display the numbers you find as `TotalPreviousAthletes`

Hint: Select the name of the schools and the sum of `PreviousAthletes`.

In [ ]:
query = """
select School, sum(PreviousAthletes) as TotalPreviousAthletes
from ncaa
where PreviousEligibility == 1 and Conference like 'The Ivy League'
group by School
"""
sql_df = pd.read_sql(query, con=db)
sql_df

,School,TotalPreviousAthletes
0,Brown University,163.0
1,Columbia University-Barnard College,75.0
2,Cornell University,98.0
3,Dartmouth College,185.0
4,Harvard University,151.0
5,Princeton University,75.0
6,University of Pennsylvania,71.0
7,Yale University,149.0


### Question 6
Find the three sports that currently have the smartest athletes. For a sport to be considered, it needs to have at least 555 athletes.

Hint: Select the sport and the average score of its teams and use a having clause.

In [ ]:
query = """
Select Sport, avg(Score) as AverageScore
from ncaa
group by Sport
having sum(Athletes) >= 555
order by AverageScore desc
limit 3
"""
sql_df = pd.read_sql(query, con=db)
sql_df

,Sport,AverageScore
0,Women's Soccer,989.181818
1,Women's Tennis,988.424242
2,Women's Basketball,985.794118


### Question 7
Find the average points by which each school, all sports combined, improved their `Score` compared to `PreviousScore`. Additionally, round that average to 2 decimals, call it `AverageScoreImprovement` and order the results by that average.

Hint: You need to group the observations by `School` and take the average of ` (Score - PreviousScore)`.

In [ ]:
query = """
select School, round(avg(Score - PreviousScore), 2) as AverageScoreImprovement
from ncaa
group by School
order by AverageScoreImprovement desc
"""
sql_df = pd.read_sql(query, con=db)
sql_df

,School,AverageScoreImprovement
0,"University of Maryland, College Park",43.60
1,University of Nebraska Omaha,38.71
2,Clemson University,29.83
3,University of Southern California,25.60
4,Syracuse University,23.33
...,...,...
63,University of Oregon,-18.00
64,Oregon State University,-25.25
65,University of Iowa,-30.20
66,Georgia Institute of Technology,-33.50


### Question 8
Rank the men's spoyrts according to their `PastPopularit`, which is the total amount of `Athletes` that played a sport.

Hint: To select only men's sports you can use `'M%'` with a `like` in a `where` clause.

In [ ]:
query = """
select Sport, sum(PreviousAthletes) as PastPopularity
from ncaa
where Sport like 'M%'
group by Sport
Order by PastPopularity desc
"""
sql_df = pd.read_sql(query, con=db)
sql_df

,Sport,PastPopularity
0,Men's Basketball,885.0
1,Men's Soccer,866.0
2,Men's Tennis,518.0
3,Men's Ice Hockey,356.0
4,Men's Skiing,61.0


### Question 9
Find how many athletes each of the 5 women's sports have gained (or lost).

Hint: To select only women's sports you can use `'W%'` with a `like` in a `where` clause.

In [ ]:
query = """
select Sport, sum(Athletes - PreviousAthletes) as GainedAthletes
from ncaa
where Sport like 'W%'
group by Sport
"""
sql_df = pd.read_sql(query, con=db)
sql_df

,Sport,GainedAthletes
0,Women's Basketball,25.0
1,Women's Ice Hockey,-15.0
2,Women's Skiing,-3.0
3,Women's Soccer,38.0
4,Women's Tennis,11.0


### Question 10
There are some schools that have adopted new sports between the last survey and the present one. Find out which schools adopted which new sports.

Hint: If a school did not offer a sport, its `PreviousAthletes`, `PreviousScore`, and `PreviousEligibility` are missing. If a school is offering a spoort, its `Athletes`, `Score`, and `Eligibility` are not missing. Use `Null`

In [ ]:
query = """
select School, Sport
from ncaa
where PreviousAthletes is Null and Athletes is not Null
"""
sql_df = pd.read_sql(query, con=db)
sql_df

,School,Sport
0,Pennsylvania State University,Men's Ice Hockey
1,Pennsylvania State University,Women's Ice Hockey
2,Western Illinois University,Women's Tennis
